In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser


DEVELOPER_KEY = input("Developer Key를 입력하세요: ")
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

Developer Key를 입력하세요: --INPUT YOUR USER KEY--


# 1. 채널 id 가져오기

In [2]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

search_response = youtube.search().list( #search : cost 100
    q = "레오제이",           # q : 검색어
    order = "relevance",    # order : 정렬방법 ex)date, rating, relevance
    part = "snippet",       # part : 필수 매개변수
    maxResults = 50         # maxResults : 결과개수
    ).execute()

channel_id = search_response['items'][0]['id']['channelId']

# 2. 채널 내 재생목록 id 가져오기

In [3]:
playlists = youtube.playlists().list(
    channelId = channel_id,
    part = "snippet",       # 동영상의 제목, 설명 카테고리 등 정보
    maxResults = 20
    ).execute()

In [4]:
import pandas as pd

playlist_id = []
playlist = []

for i in playlists['items']:
    playlist_id.append(i['id'])
    playlist.append(i['snippet']['title'])
    
df = pd.DataFrame([playlist_id, playlist]).T
df.columns = ['PlayList IDs', 'Playlist Titles']

df

,PlayList IDs,Playlist Titles
0,PLWBCdYdoqne0ejyQ8bx6-aYEbtcYNk1gA,Shorts
1,PLWBCdYdoqne21lMlDOFi6hEiDENDHLgOW,리뷰｜Review
2,PLWBCdYdoqne0xV0QVrvqiIHJSs6QYu6F4,vlog
3,PLWBCdYdoqne124Nd7rPQFzOd5YtQSSEGC,커버 메이크업｜ Cover Makeup


# 3. 재생목록 내 동영상 가져오기

## 1) 각 재생목록 내 동영상 가져오기

In [5]:
leojs = []     # 각 재생목록을 담을 리스트 선언

for n in range(len(playlist)):
    leoj = df['PlayList IDs'][n]
    playlist_videos = youtube.playlistItems().list(
        playlistId = leoj,
        part = "snippet",
        maxResults = 50
    )
    playlistitems_list_response = playlist_videos.execute()

    video_names=[]
    video_ids=[]
    date=[]

    for v in playlistitems_list_response['items']:
        video_names.append(v['snippet']['title'])
        video_ids.append(v['snippet']['resourceId']['videoId'])
        date.append(v['snippet']['publishedAt'])

    vdf = pd.DataFrame([date, video_names, video_ids]).T
    vdf.columns = ['Date', 'Title', 'IDS']

    
    #동영상 정보 가져오기
    import re
    import numpy as np
    
    category_id = []
    views = []
    likes = []
    dislikes = []
    comments = []
    mins = []
    seconds = []
    title = []
    dates = []

    for u in range(len(vdf)):
        request = youtube.videos().list(
            part = "snippet, contentDetails, statistics",
            id = vdf['IDS'][u]
        )
        response = request.execute()

        if response['items'] == []:    # 동영상 정보가 없을 경우
            video_ids.append("-")
            category_id.append("-")
            views.append("-")
            likes.append("-")
            dislikes.append("-")
            comments.append("-")
            dates.append("-")

        else:
            title.append(response['items'][0]['snippet']['title'])
            category_id.append(response['items'][0]['snippet']['categoryId'])
            dates.append(response['items'][0]['snippet']['publishedAt'].split('T')[0])
            views.append(response['items'][0]['statistics']['viewCount'])
            likes.append(response['items'][0]['statistics']['likeCount'])
            dislikes.append(response['items'][0]['statistics']['dislikeCount'])
            comments.append(response['items'][0]['statistics']['commentCount'])

    leoj_df = pd.DataFrame([dates, title, views, likes, dislikes, comments]).T
    leoj_df.columns = ['업로드일', '제목', '조회수', '좋아요', '싫어요', '댓글수']
    leoj_df.index = np.arange(1, len(leoj_df) + 1)
    leojs.append(leoj_df)

In [6]:
leojs[0]

,업로드일,제목,조회수,좋아요,싫어요,댓글수
1,2021-03-18,저 해외진출 노려봐도 될까요...?🤔 이모지 메이크업 Emoji makeupㅣ#Sh...,1823687,55923,1556,1141
2,2021-07-21,크루엘라가 된 레오제이,921640,28392,597,953
3,2021-10-26,핵불닭맛 아옳이 만들기🌶,386761,4943,152,58
4,2021-10-27,고추장 와사비맛이 된 나나 레오제이 ❤️💚,432818,8348,183,134


In [7]:
leojs[3]

,업로드일,제목,조회수,좋아요,싫어요,댓글수
1,2020-03-03,"최애란 말예요…💚 ""빌리아일리시"" 커버메이크ᄋ...",139947,4836,33,406
2,2019-10-25,🖤 Maleficent 2 makeup transformation 말레피센트 2 커...,48190,1405,39,252
3,2019-01-08,ep.8 스카이캐슬 김주영 선생님 커버 메이ᄏ...,57317,1192,50,147
4,2018-11-13,ep7. BTS 지민 커버 메이크업 BTS JIMIN Celebrity Cover ...,170353,4022,101,289
5,2018-10-12,"ep6. ""앤 해서웨이"" 커버 메이크업 Anne Hathaway Celebrity ...",78619,1384,79,133
6,2018-09-01,"ep5. ""유병재"" 커버 메이크업 Byung Jae Yoo Celebrity Cov...",58288,1095,32,114
7,2018-08-05,"ep4. ""박막례 할머니"" 커버 메이크업 Mak Rye Park Celebrity ...",225288,5950,128,696
8,2018-07-31,"ep3. ""현아"" 커버 메이크업 Hyun Ah Celebrity Cover Make...",42162,731,52,124
9,2018-07-22,ep 2.”마마무 화사” 커버 메이크업 +곱창 먹기 H...,126190,2551,122,373
10,2018-07-20,"ep1. ""문가비"" 커버 메이크업 Moon Gabi Celebr...",108778,2930,44,419


# 4. 3의 재생목록을 하나로 병합 후 엑셀 추출

In [8]:
result = leojs[0]
for n in range(1, len(leojs)):
    result = result.append(leojs[n])

In [9]:
import openpyxl
result.to_excel('./leoj_youtube.xlsx', sheet_name='youtube')